In [1]:
from frontend.database import session
from scripts import generate_fc_layer_circuit, benchmark_utils
import numpy as np

In [2]:
weight_size = 2000
weight = np.asarray([int(np.random.random() > 0.5) for i in range(weight_size)])

In [3]:
circuit_file=generate_fc_layer_circuit.generate(weight.size, weight)
print(circuit_file)

/Users/asanyal/research/SHEEP//benchmark_inputs/mid_level/circuits/circuit-fc_layer-2000.sheep


In [4]:
inp_var = benchmark_utils.get_inputs(circuit_file)
out_var = benchmark_utils.get_outputs(circuit_file)
inp_dict = {}
for inps in inp_var:
    inp_dict[inps] = int(np.random.random() > 0.5)

In [5]:
inputs_file = benchmark_utils.write_inputs_file(inp_dict)

In [6]:
results = benchmark_utils.run_circuit(circuit_file,inputs_file,"bool","TFHE")

In [7]:
assert np.alltrue([int(weight[i]==inp_dict['FCINPUT_'+str(i)]) == int(results['Outputs']['XNOROUTPUT_'+str(i)]) for i in range(weight_size)])

In [8]:
print(results['Processing times (s)']['circuit_evaluation'])

0.0049


In [9]:
from scripts.rocks import circuit, enc_vec
from scripts.particles import variables
from scripts.reusable_modules import oneb_adder, nb_adder

In [10]:
x = variables('x')
y = variables('y')
cin = variables('cin')
s = variables('sum')
c = variables('carry')
b1_adder = oneb_adder('1b_adder', [x, y, cin], [s, c], 1)
c_obj = circuit('1 Bit Adder', b1_adder)
c_obj.write_file(filename='./test.sheep')
inp_dict = {'x':1, 'y':1, 'cin':1}
inputs_file = benchmark_utils.write_inputs_file(inp_dict)
results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE")

In [11]:
print('Carry : {:d} and Sum : {:d}'.format(int(results['Outputs']['carry']),int(results['Outputs']['sum'])))

Carry : 1 and Sum : 1


In [12]:
x1 = variables('x1')
x2 = variables('x2')
x3 = variables('x3')
y1 = variables('y1')
y2 = variables('y2')
y3 = variables('y3')
cin = variables('cin')
s1 = variables('sum1')
s2 = variables('sum2')
s3 = variables('sum3')
s4 = variables('sum4')
b3_adder = nb_adder('2b_adder', [cin, x1, x2, x3, y1, y2, y3], [s1, s2, s3, s4], 3)
c_obj = circuit('3 Bit Adder', b3_adder)
c_obj.write_file(filename='./test.sheep')
inp_dict = {'x1':0, 'x2':1, 'x3': 1, 'y1':1, 'y2':1,'y3':1,'cin':0}
inputs_file = benchmark_utils.write_inputs_file(inp_dict)
results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE")

In [13]:
print('{:d}{:d}{:d}{:d}'.format(int(results['Outputs']['sum1']),
                            int(results['Outputs']['sum2']), 
                            int(results['Outputs']['sum3']), 
                            int(results['Outputs']['sum4'])
                           ))

1011


In [29]:
cin_vec = enc_vec(name='cin', nb=1, randomize_name=4)
t1 = enc_vec(name='v1', nb=3, randomize_name=4)
t2 = enc_vec(name='v2', nb=2, randomize_name=4)

In [30]:
t3 = cin + t1 + t2

TypeError: can only concatenate list (not "enc_vec") to list

In [17]:
b3_adder = nb_adder('2b_adder', t3, [s1, s2, s3, s4], 2)
c_obj = circuit('2 Bit Adder', b3_adder)
c_obj.write_file(filename='./test.sheep')

In [27]:
cin = [0]
t1_inp = [0, 0, 1]
t2_inp = [1, 1, 1]

In [28]:
inp_dict = t3.get_input_dict(cin+t1_inp+t2_inp)
inputs_file = benchmark_utils.write_inputs_file(inp_dict)
results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE")
print('{:d}{:d}{:d}'.format(int(results['Outputs']['sum1']),
                            int(results['Outputs']['sum2']), 
                            int(results['Outputs']['sum3'])
                           ))

110
